<!-- NOTEBOOK_METADATA source: "Jupyter Notebook" title: "Open Source Observability for LangGraph" description: "Learn how to use Langfuse for open source observability/tracing in your LangGraph application (Python)." category: "Integrations" -->

# 实践手册：LangGraph 集成


## 什么是 LangGraph？

[LangGraph](https://langchain-ai.github.io/langgraph/) 是由 LangChain 团队开源的框架，用于基于大语言模型（LLM）构建复杂、有状态的多智能体应用。LangGraph 内置了持久化能力，可保存与恢复状态，从而支持错误恢复与包含“人类参与”（Human-in-the-loop, HITL）的工作流。

## 本实践手册的目标

本手册演示如何借助 [Langfuse](https://langfuse.com/docs)，通过其与 [LangChain 的集成](https://langfuse.com/integrations/frameworks/langchain)，对你的 LangGraph 应用进行调试、分析与迭代优化。

**完成本手册后，你将能够：**

- 自动通过 Langfuse 集成对 LangGraph 应用进行追踪（tracing）
- 监控复杂的多智能体（multi-agent）方案
- 添加评分（例如用户反馈）
- 使用 Langfuse 管理 LangGraph 中使用的提示词（prompt）


## 初始化 Langfuse

在 Langfuse 控制台项目设置页获取你的 [API 密钥](https://langfuse.com/faq/all/where-are-langfuse-api-keys)，并将其加入到运行环境变量中以初始化 Langfuse 客户端。

<!-- CALLOUT_START type: "info" emoji: "⚠️" -->
_**注意：** 本笔记使用 Langfuse Python SDK v3。若你仍在使用 v2，请参阅我们的[旧版 LangGraph 集成指南](https://github.com/langfuse/langfuse-docs/blob/662509b3296daddcddb292f14b10a62e7c39407d/cookbook/integration_langgraph.ipynb)。_
<!-- CALLOUT_END -->

<!-- CALLOUT_START type: "info" emoji: "ℹ️" -->
_**注意：** 需要至少 Python 3.11（参见 [GitHub Issue](https://github.com/langfuse/langfuse/issues/1926)）。_
<!-- CALLOUT_END -->

In [ ]:
%pip install langfuse langchain langgraph langchain_openai langchain_community

In [ ]:
import os

# 🔑 从项目设置页面获取您的 API 密钥：https://cloud.langfuse.com
# 📋 配置步骤：
# 1. 登录 Langfuse 控制台
# 2. 选择或创建项目
# 3. 进入项目设置页面
# 4. 复制公钥和私钥

# Langfuse 公钥（用于客户端身份验证）
os.environ["LANGFUSE_PUBLIC_KEY"] = "pk-lf-..." 

# Langfuse 私钥（用于服务端身份验证，请妥善保管）
os.environ["LANGFUSE_SECRET_KEY"] = "sk-lf-..." 

# Langfuse 服务器地址（选择离您最近的区域以获得更好性能）
os.environ["LANGFUSE_HOST"] = "https://cloud.langfuse.com"  # 🇪🇺 欧洲区域
# os.environ["LANGFUSE_HOST"] = "https://us.cloud.langfuse.com"  # 🇺🇸 美国区域

# 🤖 OpenAI API 密钥
# 从 https://platform.openai.com/api-keys 获取
os.environ["OPENAI_API_KEY"] = "sk-proj-..."

# 🚨 安全提醒：
# - 不要将真实密钥提交到公共代码仓库
# - 生产环境建议使用环境变量文件（.env）或密钥管理服务
# - 定期轮换 API 密钥以提高安全性

在环境变量设置完成后，我们即可初始化 Langfuse 客户端。`get_client()` 会使用环境变量中提供的凭据来初始化 Langfuse 客户端。

In [ ]:
from langfuse import get_client
 
# 🚀 初始化 Langfuse 客户端
# get_client() 会自动读取环境变量中的配置信息
langfuse = get_client()
 
# 🔍 验证客户端连接状态
# 这个步骤非常重要，确保后续的追踪功能能够正常工作
if langfuse.auth_check():
    print("✅ Langfuse 客户端已通过身份验证，准备就绪！")
    print("🔧 现在可以开始使用追踪功能了")
else:
    print("❌ 身份验证失败！")
    print("🔍 请检查以下配置项：")
    print("   - LANGFUSE_PUBLIC_KEY 是否正确")
    print("   - LANGFUSE_SECRET_KEY 是否正确") 
    print("   - LANGFUSE_HOST 是否可访问")
    print("   - 网络连接是否正常")

## 示例 1：使用 LangGraph 构建简单聊天应用

**本节将完成：**

- 在 LangGraph 中构建一个可回答常见问题的客服聊天机器人
- 使用 Langfuse 对机器人的输入与输出进行追踪（tracing）

我们先从一个基础机器人入手，随后在下一节扩展为更高级的多智能体（multi-agent）设置，并在过程中介绍关键的 LangGraph 概念。

### 创建智能体（Agent）

首先创建一个 `StateGraph`。`StateGraph` 定义了聊天机器人的状态机结构。我们会添加节点来表示 LLM 以及机器人可调用的函数，并通过边（edge）定义机器人在这些函数之间的状态流转。

In [ ]:
# 🔧 导入 LangGraph 构建智能体所需的核心模块
from typing import Annotated
from langchain_openai import ChatOpenAI  # OpenAI 聊天模型
from langchain_core.messages import HumanMessage  # 人类消息类型
from typing_extensions import TypedDict  # 类型化字典
from langgraph.graph import StateGraph  # LangGraph 状态图
from langgraph.graph.message import add_messages  # 消息添加函数

# 📋 定义智能体的状态结构
# State 是一个类型化字典，定义了智能体在执行过程中需要维护的状态信息
class State(TypedDict):
    # 💬 消息列表：存储对话历史
    # Annotated[list, add_messages] 的含义：
    # - list: 消息的数据类型是列表
    # - add_messages: 指定状态更新策略，新消息会追加到列表末尾而不是覆盖整个列表
    # 这种设计确保了对话历史的完整保存
    messages: Annotated[list, add_messages]

# 🏗️ 创建状态图构建器
# StateGraph 是 LangGraph 的核心组件，用于定义智能体的工作流程
graph_builder = StateGraph(State)

# 🤖 初始化语言模型
# 选择 GPT-4o 模型，temperature=0.2 确保输出相对稳定但仍有一定创造性
llm = ChatOpenAI(model="gpt-4o", temperature=0.2)

# 🔄 定义聊天机器人节点函数
# 这是 LangGraph 节点函数的基本模式：接收当前状态，返回更新后的状态
def chatbot(state: State):
    """
    聊天机器人节点的核心逻辑
    
    参数:
        state (State): 当前的智能体状态，包含消息历史
    
    返回:
        dict: 包含新生成消息的状态更新
    
    工作流程:
    1. 获取当前的消息历史
    2. 将消息历史发送给语言模型
    3. 接收模型生成的回复
    4. 将回复包装成状态更新返回
    """
    # 调用语言模型处理当前对话历史，生成回复
    response = llm.invoke(state["messages"])
    
    # 返回状态更新：将模型的回复添加到消息列表中
    return {"messages": [response]}

# 🔗 向图中添加"chatbot"节点
# 节点代表工作单元，通常是普通的 Python 函数
# 每个节点负责特定的处理逻辑，如调用 LLM、处理工具、数据转换等
graph_builder.add_node("chatbot", chatbot)

# 🚀 设置图的入口点
# 告诉图每次运行时从哪个节点开始执行
# 在这个简单示例中，我们直接从 chatbot 节点开始
graph_builder.set_entry_point("chatbot")

# 🏁 设置图的结束点
# 指示图"当这个节点运行完成后，可以退出执行"
# 对于简单的单轮对话，chatbot 节点执行完就可以结束
graph_builder.set_finish_point("chatbot")

# ⚙️ 编译图形为可执行对象
# compile() 方法将图构建器转换为 CompiledGraph
# CompiledGraph 是可以实际运行的图形对象，支持 invoke、stream 等方法
graph = graph_builder.compile()

# 💡 理解 LangGraph 的核心概念：
# 🏗️ StateGraph: 定义智能体的状态和工作流程
# 🔄 Node: 执行具体任务的函数，如调用 LLM、使用工具等
# 🔗 Edge: 连接节点，定义执行顺序和条件跳转
# 📊 State: 智能体运行过程中维护的数据结构
# ⚙️ CompiledGraph: 编译后的可执行图形对象

### 在调用时添加 Langfuse 回调

现在，为了追踪应用执行过程，我们将添加 [面向 LangChain 的 Langfuse 回调处理器](https://langfuse.com/integrations/frameworks/langchain)：`config={"callbacks": [langfuse_handler]}`

In [ ]:
# 🔧 导入 Langfuse 的 LangChain 回调处理器
from langfuse.langchain import CallbackHandler

# 🔄 初始化 Langfuse 回调处理器
# 这个处理器将自动捕获 LangChain/LangGraph 的执行过程，实现以下功能：
# - 🕒 记录每个节点的执行时间和延迟
# - 📝 捕获输入输出数据和中间状态
# - 💰 跟踪 token 使用量和 API 调用成本
# - 🐛 记录错误信息和异常堆栈（如果有）
# - 📊 生成完整的执行链路追踪图
langfuse_handler = CallbackHandler()

# 🚀 运行智能体并启用 Langfuse 追踪
# 示例对话：询问 Langfuse 的相关信息
print("🤖 智能体开始运行，正在处理问题...")
print("❓ 问题：什么是 Langfuse？")
print("\n📋 执行过程：")

# 使用 stream 方法可以实时查看智能体的执行步骤
# graph.stream() 返回每个节点执行后的状态快照
for step_result in graph.stream(
    # 输入状态：包含用户的问题
    {"messages": [HumanMessage(content="什么是 Langfuse？请详细介绍其主要功能和应用场景。")]},
    # 配置回调处理器以启用 Langfuse 追踪
    config={"callbacks": [langfuse_handler]}
):
    print(f"📤 节点执行结果: {step_result}")

print("\n✅ 智能体执行完成！")
print("🔍 您可以在 Langfuse 控制台中查看完整的执行追踪记录")

### 在 Langfuse 中查看追踪结果

示例追踪：`https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/85b0c53c4414f22ed8bfc9eb35f917c4`


![在 Langfuse 中查看聊天应用的追踪](https://langfuse.com/images/cookbook/integration-langgraph/integration_langgraph_chatapp_trace.png)

### 可视化聊天应用

你可以使用 `get_graph` 方法配合相应的 “draw” 方法对图进行可视化。

In [ ]:
from IPython.display import Image, display
display(Image(graph.get_graph().draw_mermaid_png()))

```mermaid
graph TD;
	__start__([__start__]):::first
	chatbot(chatbot)
	__end__([__end__]):::last
	__start__ --> chatbot;
	chatbot --> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc
```

### 在 LangGraph Server 中使用 Langfuse

#### 🖥️ 什么是 LangGraph Server？

[LangGraph Server](https://langchain-ai.github.io/langgraph/concepts/langgraph_server/) 是 LangGraph 的服务器部署解决方案，它提供：

- 🌐 **HTTP API 接口**：将 LangGraph 智能体包装为 REST API 服务
- 🚀 **生产就绪**：支持高并发、负载均衡和容器化部署
- 🔧 **简化运维**：自动处理请求路由、状态管理和错误处理
- 📊 **监控集成**：原生支持各种监控和追踪工具
- 🔒 **安全性**：内置身份验证和授权机制

#### 💡 为什么在 Server 中使用 Langfuse？

在服务器环境中，Langfuse 追踪变得更加重要：

- 🏭 **生产监控**：实时监控服务器上智能体的运行状态
- 🐛 **远程调试**：无需本地复现即可分析线上问题
- 📈 **性能分析**：识别性能瓶颈和优化机会
- 💰 **成本控制**：精确跟踪 API 使用量和费用
- 👥 **团队协作**：多人共享追踪数据和分析结果

#### 🔧 配置方法说明

使用 LangGraph Server 时，智能体图的调用由服务器自动处理，用户无法在每次请求时手动指定回调处理器。

**关键差异：**
- 🏠 **本地开发**：可以在每次调用时添加 `config={"callbacks": [langfuse_handler]}`
- 🖥️ **服务器部署**：需要在图编译时预先配置回调处理器

**解决方案：**
在声明和编译图时就添加 Langfuse 回调，这样服务器上的所有请求都会自动启用追踪功能。

In [ ]:
# 🔧 导入服务器部署所需的模块
from typing import Annotated
from langchain_openai import ChatOpenAI
from typing_extensions import TypedDict
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langfuse.langchain import CallbackHandler

# 📋 定义智能体状态（与之前相同）
class State(TypedDict):
    messages: Annotated[list, add_messages]

# 🏗️ 构建图形结构
graph_builder = StateGraph(State)

# 🤖 初始化语言模型
llm = ChatOpenAI(model="gpt-4o", temperature=0.2)

# 🔄 定义聊天机器人节点
def chatbot(state: State):
    """处理用户消息并生成回复"""
    return {"messages": [llm.invoke(state["messages"])]}

# 🔗 组装图形结构
graph_builder.add_node("chatbot", chatbot)
graph_builder.set_entry_point("chatbot")
graph_builder.set_finish_point("chatbot")

# 🔄 初始化 Langfuse 回调处理器（服务器模式）
# 在服务器环境中，这个处理器会自动处理所有请求的追踪
langfuse_handler = CallbackHandler()

# ⚙️ 编译图形并预配置回调处理器
# 🎯 关键方法：with_config()
# - compile(): 编译图形为可执行对象
# - with_config(): 为编译后的图形添加默认配置
# 
# 💡 工作原理：
# 1. 首先编译图形，得到 CompiledGraph 对象
# 2. 然后使用 with_config() 为图形设置默认回调
# 3. 返回新的 CompiledGraph，内置了 Langfuse 追踪功能
# 
# 🚀 优势：
# - 无需在每次调用时手动添加回调配置
# - 服务器上的所有请求都会自动启用追踪
# - 简化了生产环境的部署和维护
graph = graph_builder.compile().with_config({"callbacks": [langfuse_handler]})

# 💡 部署提示：
# 在 LangGraph Server 中，可以直接使用这个预配置的 graph 对象
# 所有通过 API 发送的请求都会自动记录到 Langfuse 中

## 示例 2：基于 LangGraph 的多智能体应用

**本节将完成：**

- 构建 2 个执行智能体：一个研究智能体使用 LangChain 的 WikipediaAPIWrapper 搜索维基百科，另一个使用自定义工具获取当前时间
- 构建一个智能体监督者（supervisor），用于将用户问题分配给上述智能体
- 添加 Langfuse 回调以追踪监督者与执行智能体的步骤

In [ ]:
%pip install langfuse langgraph langchain langchain_openai langchain_experimental pandas wikipedia

### 创建工具

在本示例中，我们将构建一个用于维基百科检索的智能体，以及一个用于告知当前时间的智能体。先定义它们将使用的工具：

In [ ]:
# 🔧 导入多智能体系统所需的工具和模块
from typing import Annotated
from langchain_community.tools import WikipediaQueryRun  # 维基百科查询工具
from langchain_community.utilities import WikipediaAPIWrapper  # 维基百科 API 封装
from datetime import datetime  # 时间处理模块
from langchain.tools import Tool  # 通用工具定义类

# 🔍 定义维基百科搜索工具
# 功能：根据查询词在维基百科中搜索相关信息
# 适用场景：回答百科知识、历史事件、人物传记等问题
wikipedia_tool = WikipediaQueryRun(
    api_wrapper=WikipediaAPIWrapper(
        top_k_results=2,  # 返回最相关的2个搜索结果
        doc_content_chars_max=1000  # 限制每个结果的字符数，避免信息过载
    )
)

# ⏰ 定义当前时间查询工具  
# 功能：返回当前的日期和时间信息
# 适用场景：回答"现在几点"、"今天是什么日期"等时间相关问题
datetime_tool = Tool(
    name="Datetime",  # 工具名称，智能体会通过这个名称调用工具
    func=lambda x: datetime.now().isoformat(),  # 工具函数：返回 ISO 格式的当前时间
    description="返回当前的日期和时间信息（ISO 格式）",  # 工具描述，帮助智能体理解何时使用此工具
)

# 💡 工具设计原则：
# 1. 🎯 单一职责：每个工具只负责一个特定功能
# 2. 📝 清晰描述：description 要准确描述工具的功能和使用场景
# 3. 🔒 错误处理：生产环境中应该添加异常处理逻辑
# 4. ⚡ 性能考虑：限制返回数据的大小，避免影响整体性能

### 🛠️ 辅助工具函数

#### 📝 功能说明
下面定义的辅助函数用于简化添加新的智能体工作节点。这些函数封装了创建智能体和节点的通用逻辑，提高代码的可重用性和可维护性。

#### 🎯 设计目标
- **减少重复代码**：避免为每个智能体重复编写相同的初始化逻辑
- **标准化接口**：确保所有智能体节点具有一致的输入输出格式
- **简化扩展**：新增智能体时只需关注业务逻辑，无需处理框架细节

In [ ]:
# 🔧 导入智能体构建所需的核心组件
from langchain.agents import AgentExecutor, create_openai_tools_agent  # 智能体执行器和创建函数
from langchain_core.messages import BaseMessage, HumanMessage  # 消息基类和人类消息
from langchain_openai import ChatOpenAI  # OpenAI 聊天模型

def create_agent(llm: ChatOpenAI, system_prompt: str, tools: list):
    """
    🏭 智能体工厂函数：创建具有特定能力的工作智能体
    
    参数:
        llm (ChatOpenAI): 语言模型实例
        system_prompt (str): 系统提示词，定义智能体的角色和行为规范
        tools (list): 智能体可使用的工具列表
    
    返回:
        AgentExecutor: 配置完成的智能体执行器
    
    🔄 工作流程:
    1. 构建提示模板（包含系统角色、对话历史、工具使用记录）
    2. 创建支持工具调用的 OpenAI 智能体
    3. 包装为执行器，处理工具调用和状态管理
    """
    # 📋 构建智能体的提示模板
    # 包含三个关键部分：系统角色、对话历史、工具使用记录
    prompt = ChatPromptTemplate.from_messages([
        # 🎭 系统消息：定义智能体的角色、能力和行为规范
        ("system", system_prompt),
        # 💬 消息历史：保存与用户和其他智能体的对话记录
        MessagesPlaceholder(variable_name="messages"),
        # 🔧 工具记录：记录智能体使用工具的过程和结果
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ])
    
    # 🤖 创建支持 OpenAI 工具调用的智能体
    # 这个智能体能够理解何时需要使用工具，以及如何解释工具的返回结果
    agent = create_openai_tools_agent(llm, tools, prompt)
    
    # 🎮 创建智能体执行器
    # 执行器负责：工具调用、错误处理、状态管理、结果整合
    executor = AgentExecutor(
        agent=agent,
        tools=tools,
        verbose=True,  # 显示详细的执行过程，便于调试
        handle_parsing_errors=True  # 自动处理解析错误，提高稳定性
    )
    
    return executor

def agent_node(state, agent, name):
    """
    🔄 智能体节点适配器：将智能体包装为 LangGraph 节点
    
    参数:
        state: 当前的图状态，包含消息历史和其他上下文信息
        agent: 智能体执行器实例
        name: 智能体的名称，用于在多智能体系统中标识消息来源
    
    返回:
        dict: 包含智能体响应的状态更新
    
    🔄 适配过程:
    1. 调用智能体处理当前状态
    2. 提取智能体的输出结果
    3. 包装为带有发送者身份的消息
    4. 返回状态更新
    """
    # 📤 调用智能体处理当前状态
    # agent.invoke() 会处理对话历史，决定是否使用工具，并生成最终回复
    result = agent.invoke(state)
    
    # 🏷️ 将智能体的输出包装为带有身份标识的消息
    # name 参数让系统知道这条消息来自哪个智能体
    return {
        "messages": [HumanMessage(
            content=result["output"],  # 智能体生成的文本内容
            name=name  # 消息发送者的身份标识
        )]
    }

### 🎯 创建智能体监督者

#### 📋 监督者的核心职责
智能体监督者是多智能体系统的"大脑"，负责：

- 🧠 **任务理解**：分析用户请求，理解任务的性质和需求
- 🎯 **智能体选择**：根据任务特点选择最适合的工作智能体
- 🔄 **流程控制**：决定何时切换智能体，何时结束处理流程
- 📊 **结果整合**：汇总各个智能体的工作成果

#### 🔧 技术实现方式
监督者使用 **函数调用（Function Calling）** 技术来实现决策：

- 📞 **函数调用**：通过结构化的函数调用来表达决策结果
- 🎛️ **选择机制**：在可用的工作节点中选择下一个执行者
- 🏁 **终止条件**：判断何时任务已完成，可以结束处理流程

#### 💡 设计优势
- **精确控制**：避免随机或不确定的路由决策
- **可解释性**：每个决策都有明确的逻辑依据
- **可扩展性**：容易添加新的工作智能体和决策规则

In [ ]:
from langchain_core.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

members = ["Researcher", "CurrentTime"]
system_prompt = (
    "You are a supervisor tasked with managing a conversation between the"
    " following workers:  {members}. Given the following user request,"
    " respond with the worker to act next. Each worker will perform a"
    " task and respond with their results and status. When finished,"
    " respond with FINISH."
)
# Our team supervisor is an LLM node. It just picks the next agent to process and decides when the work is completed
options = ["FINISH"] + members

# Using openai function calling can make output parsing easier for us
function_def = {
    "name": "route",
    "description": "Select the next role.",
    "parameters": {
        "title": "routeSchema",
        "type": "object",
        "properties": {
            "next": {
                "title": "Next",
                "anyOf": [
                    {"enum": options},
                ],
            }
        },
        "required": ["next"],
    },
}

# Create the prompt using ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        (
            "system",
            "Given the conversation above, who should act next?"
            " Or should we FINISH? Select one of: {options}",
        ),
    ]
).partial(options=str(options), members=", ".join(members))

llm = ChatOpenAI(model="gpt-4o")

# Construction of the chain for the supervisor agent
supervisor_chain = (
    prompt
    | llm.bind_functions(functions=[function_def], function_call="route")
    | JsonOutputFunctionsParser()
)

### Construct graph

Now we are ready to start building the graph. Below, define the state and worker nodes using the function we just defined. Then we connect all the edges in the graph.

In [12]:
import functools
import operator
from typing import Sequence, TypedDict
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import END, StateGraph, START

# The agent state is the input to each node in the graph
class AgentState(TypedDict):
    # The annotation tells the graph that new messages will always be added to the current states
    messages: Annotated[Sequence[BaseMessage], operator.add]
    # The 'next' field indicates where to route to next
    next: str

# Add the research agent using the create_agent helper function
research_agent = create_agent(llm, "You are a web researcher.", [wikipedia_tool])
research_node = functools.partial(agent_node, agent=research_agent, name="Researcher")

# Add the time agent using the create_agent helper function
currenttime_agent = create_agent(llm, "You can tell the current time at", [datetime_tool])
currenttime_node = functools.partial(agent_node, agent=currenttime_agent, name = "CurrentTime")

workflow = StateGraph(AgentState)

# Add a "chatbot" node. Nodes represent units of work. They are typically regular python functions.
workflow.add_node("Researcher", research_node)
workflow.add_node("CurrentTime", currenttime_node)
workflow.add_node("supervisor", supervisor_chain)

# We want our workers to ALWAYS "report back" to the supervisor when done
for member in members:
    workflow.add_edge(member, "supervisor")

# Conditional edges usually contain "if" statements to route to different nodes depending on the current graph state.
# These functions receive the current graph state and return a string or list of strings indicating which node(s) to call next.
conditional_map = {k: k for k in members}
conditional_map["FINISH"] = END
workflow.add_conditional_edges("supervisor", lambda x: x["next"], conditional_map)

# Add an entry point. This tells our graph where to start its work each time we run it.
workflow.add_edge(START, "supervisor")

# To be able to run our graph, call "compile()" on the graph builder. This creates a "CompiledGraph" we can use invoke on our state.
graph_2 = workflow.compile()

### Add Langfuse as callback to the invocation

Add [Langfuse handler](https://langfuse.com/integrations/frameworks/langchain) as callback: `config={"callbacks": [langfuse_handler]}`

In [ ]:
from langfuse.langchain import CallbackHandler

# Initialize Langfuse CallbackHandler for Langchain (tracing)
langfuse_handler = CallbackHandler()

# Add Langfuse handler as callback: config={"callbacks": [langfuse_handler]}
# You can also set an optional 'run_name' that will be used as the trace name in Langfuse
for s in graph_2.stream({"messages": [HumanMessage(content = "How does photosynthesis work?")]},
                      config={"callbacks": [langfuse_handler]}):
    print(s)
    print("----")

In [ ]:
# Add Langfuse handler as callback: config={"callbacks": [langfuse_handler]}
for s in graph_2.stream({"messages": [HumanMessage(content = "What time is it?")]},
                      config={"callbacks": [langfuse_handler]}):
    print(s)
    print("----")

### See traces in Langfuse

Example traces in Langfuse:

1. [How does photosynthesis work?](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/7d5f970573b8214d1ca891251e42282c)
2. [What time is it?](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/3a69fe4998df50d42054f8944bd6a8d9)

![在 Langfuse 中查看多智能体追踪](https://langfuse.com/images/cookbook/integration-langgraph/integration_langgraph_multiagent_traces.png)

### 可视化该智能体

你可以使用 `get_graph` 方法配合相应的 “draw” 方法进行图形可视化。

In [ ]:
from IPython.display import Image, display
display(Image(graph_2.get_graph().draw_mermaid_png()))

```mermaid
graph TD;
	__start__([__start__]):::first
	Researcher(Researcher)
	CurrentTime(CurrentTime)
	supervisor(supervisor)
	__end__([__end__]):::last
	CurrentTime --> supervisor;
	Researcher --> supervisor;
	__start__ --> supervisor;
	supervisor -.-> Researcher;
	supervisor -.-> CurrentTime;
	supervisor -. &nbspFINISH&nbsp .-> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc
```

## Multiple LangGraph Agents

There are setups where one LangGraph agent uses one or multiple other LangGraph agents. To combine all corresponding spans in one single trace for the multi agent execution, we can pass a custom `trace_id`. 

First, we generate a trace_id that can be used for both agents to group the agent executions together in one Langfuse trace.

In [14]:
from langfuse import get_client, Langfuse
from langfuse.langchain import CallbackHandler
 
langfuse = get_client()
 
# Generate deterministic trace ID from external system
predefined_trace_id = Langfuse.create_trace_id()

# Initialize Langfuse CallbackHandler for Langchain (tracing)
langfuse_handler = CallbackHandler()

Next, we set up the sub-agent.

In [15]:
from typing import Annotated
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from typing_extensions import TypedDict
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages

class State(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

llm = ChatOpenAI(model = "gpt-4o", temperature = 0.2)

def chatbot(state: State):
    return {"messages": [llm.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)
graph_builder.set_entry_point("chatbot")
graph_builder.set_finish_point("chatbot")
sub_agent = graph_builder.compile()

Then, we set the tool that uses the research-sub-agent to answer questions.

In [16]:
from langchain_core.tools import tool

@tool
def langgraph_research(question):
  """Conducts research for various topics."""

  with langfuse.start_as_current_span(
      name="🤖-sub-research-agent",
      trace_context={"trace_id": predefined_trace_id}
  ) as span:
      span.update_trace(input=question)

      response = sub_agent.invoke({"messages": [HumanMessage(content = question)]},
                        config={"callbacks": [langfuse_handler]})
    
      span.update_trace(output= response["messages"][1].content)

  return response["messages"][1].content

Set up a second simple LangGraph agent that uses the new `langgraph_research`. 

In [17]:
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model = "gpt-4o", temperature = 0.2)

main_agent = create_react_agent(
    model=llm,
    tools=[langgraph_research]
)

In [ ]:
user_question = "What is Langfuse?"

# Use the predefined trace ID with trace_context
with langfuse.start_as_current_span(
    name="🤖-main-agent",
    trace_context={"trace_id": predefined_trace_id}
) as span:
    span.update_trace(input=user_question)
 
    # LangChain execution will be part of this trace
    response = main_agent.invoke({"messages": [{"role": "user", "content": user_question}]},
                            config={"callbacks": [langfuse_handler]})

    span.update_trace(output=response["messages"][1].content)
 
print(f"Trace ID: {predefined_trace_id}")  # Use this for scoring later

### View traces in Langfuse

![multi agent trace in Langfuse](https://langfuse.com/images/cookbook/integration-langgraph/a2a_langgraph.png)

Example trace in Langfuse: https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/85b0c53c4414f22ed8bfc9eb35f917c4

## Adding scores to traces as scores

[Scores](https://langfuse.com/docs/scores/overview) are used to evaluate single observations or entire traces. They enable you to implement custom quality checks at runtime or facilitate human-in-the-loop evaluation processes.

In the example below, we demonstrate how to score a specific span for `relevance` (a numeric score) and the overall trace for `feedback` (a categorical score). This helps in systematically assessing and improving your application.

**→ Learn more about [Custom Scores in Langfuse](https://langfuse.com/docs/scores/custom).**

In [19]:
from langfuse import get_client
 
langfuse = get_client()
 
# Option 1: Use the yielded span object from the context manager
with langfuse.start_as_current_span(
    name="langgraph-request") as span:
    # ... LangGraph execution ...
 
    # Score using the span object
    span.score_trace(
        name="user-feedback",
        value=1,
        data_type="NUMERIC",
        comment="This was correct, thank you"
    )
 
# Option 2: Use langfuse.score_current_trace() if still in context
with langfuse.start_as_current_span(name="langgraph-request") as span:
    # ... LangGraph execution ...
 
    # Score using current context
    langfuse.score_current_trace(
        name="user-feedback",
        value=1,
        data_type="NUMERIC"
    )
 
# Option 3: Use create_score() with trace ID (when outside context)
langfuse.create_score(
    trace_id=predefined_trace_id,
    name="user-feedback",
    value=1,
    data_type="NUMERIC",
    comment="This was correct, thank you"
)

### View trace with score in Langfuse

Example trace: https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/e60a078b828d4fdc7ea22c73193b0fe4

![Trace view including added score](https://langfuse.com/images/cookbook/integration-langgraph/integration_langgraph_score.png)

## Manage prompts with Langfuse

Use [Langfuse prompt management](https://langfuse.com/docs/prompts/example-langchain) to effectively manage and version your prompts. We add the prompt used in this example via the SDK. In production, however, users would update and manage the prompts via the Langfuse UI instead of using the SDK.

Langfuse prompt management is basically a Prompt CMS (Content Management System). Alternatively, you can also edit and version the prompt in the Langfuse UI.

*   `Name` that identifies the prompt in Langfuse Prompt Management
*   Prompt with prompt template incl. `{{input variables}}`
*   `labels` to include `production` to immediately use prompt as the default

In this example, we create a system prompt for an assistant that translates every user message into Spanish.

In [ ]:
from langfuse import get_client
 
langfuse = get_client()

langfuse.create_prompt(
    name="translator_system-prompt",
    prompt="You are a translator that translates every input text into Spanish.",
    labels=["production"]
)

![View prompt in Langfuse UI](https://langfuse.com/images/cookbook/integration-langgraph/integration_langgraph_prompt_example.png)

Use the utility method `.get_langchain_prompt()` to transform the Langfuse prompt into a string that can be used in Langchain.


**Context:** Langfuse declares input variables in prompt templates using double brackets (`{{input variable}}`). Langchain uses single brackets for declaring input variables in PromptTemplates (`{input variable}`). The utility method `.get_langchain_prompt()` replaces the double brackets with single brackets. In this example, however, we don't use any variables in our prompt.

In [ ]:
# Get current production version of prompt and transform the Langfuse prompt into a string that can be used in Langchain
langfuse_system_prompt = langfuse.get_prompt("translator_system-prompt")
langchain_system_prompt = langfuse_system_prompt.get_langchain_prompt()

print(langchain_system_prompt)

Now we can use the new system prompt string to update our assistant.

In [22]:
from typing import Annotated
from langchain_openai import ChatOpenAI
from typing_extensions import TypedDict
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages

class State(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

llm = ChatOpenAI(model = "gpt-4o", temperature = 0.2)

# Add the system prompt for our translator assistent
system_prompt = {
    "role": "system",
    "content": langchain_system_prompt
}

def chatbot(state: State):
    messages_with_system_prompt = [system_prompt] + state["messages"]
    response = llm.invoke(messages_with_system_prompt)
    return {"messages": [response]}

graph_builder.add_node("chatbot", chatbot)
graph_builder.set_entry_point("chatbot")
graph_builder.set_finish_point("chatbot")
graph = graph_builder.compile()

In [ ]:
from langfuse.langchain import CallbackHandler

# Initialize Langfuse CallbackHandler for Langchain (tracing)
langfuse_handler = CallbackHandler()

# Add Langfuse handler as callback: config={"callbacks": [langfuse_handler]}
for s in graph.stream({"messages": [HumanMessage(content = "What is Langfuse?")]},
                      config={"callbacks": [langfuse_handler]}):
    print(s)

## Add custom spans to a LangGraph trace

Sometimes it is helpful to add custom spans to a LangGraph trace. This [GitHub discussion thread](https://github.com/orgs/langfuse/discussions/2988#discussioncomment-11634600) provides an example of how to do this.